# IDAES Flowsheet Visualizer Tutorial
> Author: Dan Gunter \
> Created: 2022-07-05

## Introduction
The IDAES Flowsheet Visualizer (FV) is a Python tool that provides a web-based visualization of any existing IDAES model or flowsheet. The visualization shows a diagram of the
flowsheet and the stream table. You can interact with the diagram and export
it as an image for inclusion in presentations or publications.

This tutorial will show the basic steps of running the FV on an example
flowsheet, interacting with the resulting GUI, saving your work, and exporting
the diagram as an image. It will also show how the Visualizer can be updated
to reflect changes in the model components and/or variable values. The tutorial
will also show how to run the Visualizer from a Python script.

## Setup
Module imports and any additional housekeeping needed
to initialize the code.

In [1]:
import visualizer_tutorial as vistut
vistut.quiet()  # turn off default logging and most warnings
from idaes.core.util.model_statistics import degrees_of_freedom
from IPython.display import display, Markdown

    idaes.models.unit_models  (deprecated in 2.0.0.alpha0) (called from
    <frozen importlib._bootstrap>:228)
    has been moved to idaes.models.unit_models.pressure_changer  (deprecated
    in 2.0.0.alpha0) (called from <frozen importlib._bootstrap>:228)


## Example flowsheet

In [2]:


# use the pre-defined function to create the flowsheet
model = vistut.create_model()

# description of the flowsheet we created
display(Markdown(vistut.create_model.__doc__))

vistut.quiet()

# initialize the flowsheet as a square problem (dof=0)
vistut.initialize_model(model)

# verify that there are zero degrees of freedom
print(f"DOF = {degrees_of_freedom(model)}")

## Create an IDAES flowsheet for hydrodealkylation (HDA)

### About HDA

Hydrodealkylation (HDA) is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**

This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will not cover in this example.

### References

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.
    

DOF = 0


## Running the Flowsheet Visualizer
In most cases, you will run the FV by calling the `visualize()` method attached to your flowsheet.
This function takes a number of optional arguments, which we will look at briefly later, and one required argument:
the **title** to give the visualization. Unless you give more information, this title also is used as the filename in which to save its current state.

In the following, we start the FV with the title "Hydrodealkylation". This will pop up a new browser tab (and save the status in a file called _Hydrodealkylation.json_).

<div class="alert alert-block alert-info">
After the visualizer starts, we recommend making its tab into its own browser window and viewing it side-by-side with this notebook window.
</div>

In [3]:
model.fs.visualize("Hydrodealkylation")

Started visualization server
Loading saved flowsheet from 'Hydrodealkylation.json'
Saving flowsheet to default file 'Hydrodealkylation.json' in current directory (C:\Users\DanGunter\src\idaes\dangunter\examples-pse\src\Tutorials\Basics)
Flowsheet visualization at: http://localhost:64903/app?id=Hydrodealkylation


## Interacting with the visualizer
The first things you need to learn about the FV are how to manipulate the overall layout and control the view.
The UI should initially look something like the screenshot below:
<img src="fv1.png" alt="Screenshot of Flowsheet Visualizer">&nbsp;</img>

<div class="alert alert-block alert-info">
    As you can see, the FV has two main panels. We will call the top panel the <b>diagram</b> and the bottom panel the <b>stream table</b>.
</div>

### View controls
Before looking at the two panels in detail, it helps to know some basic controls for making them easier to view.

| Control | Description | Illustration |
|:----|:---------------------|:----:|
| Panel height | Change the height of the  panels by grabbing the small handle in the lower right corner with your mouse. | <img src="fv2.png"  alt="Handle used to resize panels" width="150px">&nbsp;</img> |
| Diagram size | Zoom in/out on the diagram with the magnifying glass "+" and "-" buttons in the upper-right corner of the top panel. The button labeled with two crossing arrows fits the diagram into the current panel height and width. | <img src="fv3.png" width="150px" alt="Handle used to resize panels"></img> |

## Saving and loading

## Exporting images

## Updating when the flowsheet changes
Below is some IDAES modeling code that adds another Flash unit to the model, connecting the liquid outlet of the first flash unit to its inlet. There is a little more code that updates some of the output values of the model and sets initial values for this new unit, and then re-initializes the model.

**After this code executes, the model will have a unit called "F102" connected to "F101".**

In [4]:
# Add a second flash unit
from idaes.generic_models.unit_models import Flash
from pyomo.network import Arc
from pyomo.environ import Expression, TransformationFactory
m = model # alias
m.fs.F102 = Flash(default={"property_package": m.fs.thermo_params,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})
# connect to 1st flash unit
m.fs.s10 = Arc(source=m.fs.F101.liq_outlet, destination=m.fs.F102.inlet)
# update expressions for purity and cost
m.fs.purity = Expression(
        expr=m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] /
             (m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"]
              + m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "toluene"]))
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0] +
                                1.9e-7 * m.fs.F102.heat_duty[0])
# fix unit output and pressure drop
m.fs.F102.vap_outlet.temperature.fix(375)
m.fs.F102.deltaP.fix(-200000)

# expand arcs
TransformationFactory("network.expand_arcs").apply_to(m)
# re-initialize
vistut.initialize_model(m)

<div class="alert alert-block alert-info">Since the FV is connected to the current state of the model in memory, simply hitting "Refresh" in the FV window will show the new flash unit in the diagram, and the new stream (liquid) in the stream table. We can then interactively rearrange the unit to be in the position we want in the diagram.</div>

In [7]:
# XXX Should not need to re-run!!
model.fs.visualize("Hydrodealkylation")

Loading saved flowsheet from 'Hydrodealkylation.json'
Saving flowsheet to default file 'Hydrodealkylation.json' in current directory (C:\Users\DanGunter\src\idaes\dangunter\examples-pse\src\Tutorials\Basics)
Flowsheet visualization at: http://localhost:64903/app?id=Hydrodealkylation


In [ ]:
# Solve the model
# Create the solver object
from pyomo.environ import SolverFactory
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6, 'max_iter': 5000}

# Solve the model
results = solver.solve(model, tee=False)

## Running the visualizer from a script
The Visualizer can be used from a Jupyter Notebook like this, or from a
Python script. The code to run this same tutorial as a Python script
is also in the file [visualizer_tutorial.py](file://visualizer_tutorial.py)
in the same folder as this notebook.

If you have installed the IDAES examples (using `idaes get-examples`),
then you can do the following to import and run the module:
```
from idaes_examples.Tutorials.Basics import visualizer_tutorial
visualizer_tutorial.main()
```



In [5]:
from pyomo.environ import TransformationFactory
TransformationFactory??